In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

from pydantic import BaseModel
from datetime import datetime
from pprint import pprint

In [5]:
class Scene(BaseModel):
    name: str
    location_tags: list[str]
    time_tags: list[str]
    other_tags: list[str]

class Scenes(BaseModel):
    scenes: list[Scene]

scenes_list = ["출퇴근길", "휴식", "공부", "게임", "유튜브 시청", "애완동물 돌보기기"]

In [6]:
class User(BaseModel):
    name: str
    location: str
    birthdate: datetime
    occupation: str
    personality: list[str]
    scenes: list[Scene] = []
    prompt: str
    positives: list[str]
    negatives: list[str]
    
user_context = {
    'name': '윤형석',
    'location': '서울',
    'birthdate': datetime.strptime('1990-03-28', '%Y-%m-%d'),
    'occupation': '예비창업자',
    'personality': ['Introverted', 'Intuitive', 'Thinking', 'Perceiving'],
    'prompt': 'Your prompt here',
    'positives': ['지적 호기심', '창의력', '직관성'],
    'negatives': ['계획성 부족', '조직력 부족', '무기력함']
}

user = User(**user_context)

user_bio = user.model_dump_json()

In [7]:
pydantic_parser = PydanticOutputParser(pydantic_object=Scenes)

scene_parser = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.5)

scene_prompt = ChatPromptTemplate.from_template("""
다음은 사용자의 하루를 구성하는 장면들입니다.
사용자의 정보와 이 장면들을 바탕으로, 각각의 장면에 어울리는 시간, 공간, 기타 태그를 3~5개씩 필요에 따라 작성하고, 아래 지침에 따라 json 포맷으로 반환하세요.

태그를 붙이는 목적은 할 일을 관리하기 위한 데이터베이스에 사용하기 위해서입니다.
각각의 할 일은 사용자의 하루를 나타내는 여러 장면을 담은 태그와 함께 저장되고, 사용자가 처한 맥락과 상황을 표현하는 태그에 맞춰 할 일을 추천합니다.

시간 태그에는 휴일 여부, 요일, 하루 중의 시간대 등의 정보를 포함하세요.
공간 태그에는 사용자의 위치, 활동하는 장소 등의 정보를 포함하세요.
기타 태그에는 시간과 공간 태그에 포함되지 않지만 할 일의 맥락과 상황을 검색하기에 좋은 정보를 포함하세요.

사용자 정보: {bio}
장면: {scenes}
지침: {format_instruction}
""").partial(format_instruction=pydantic_parser.get_format_instructions())

scenes_chain = scene_prompt | scene_parser | pydantic_parser
response = scenes_chain.invoke({"bio": user_bio, "scenes": scenes_list})

for scene in response.scenes:
    user.scenes.append(scene)
    print(scene.model_dump_json())

{"name":"출퇴근길","location_tags":["서울","지하철","버스"],"time_tags":["평일","오전","오후"],"other_tags":["통근","대중교통","시간 관리"]}
{"name":"휴식","location_tags":["집","카페"],"time_tags":["주말","오후","저녁"],"other_tags":["스트레스 해소","재충전","여유"]}
{"name":"공부","location_tags":["집","도서관"],"time_tags":["평일","오후","저녁"],"other_tags":["자기계발","지식 확장","집중"]}
{"name":"게임","location_tags":["집"],"time_tags":["주말","오후","저녁"],"other_tags":["취미","오락","사회적 상호작용"]}
{"name":"유튜브 시청","location_tags":["집","카페"],"time_tags":["주말","오후","저녁"],"other_tags":["정보 소비","여가","트렌드"]}
{"name":"애완동물 돌보기","location_tags":["집"],"time_tags":["평일","오전","오후"],"other_tags":["책임감","애정","일상"]}


In [8]:
import json

user_bio = user.model_dump_json()
user_dict = json.loads(user_bio)
user_dict['scenes'][4]

{'name': '유튜브 시청',
 'location_tags': ['집', '카페'],
 'time_tags': ['주말', '오후', '저녁'],
 'other_tags': ['정보 소비', '여가', '트렌드']}

In [9]:
# LLM 초기화
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0, max_tokens=None)

In [10]:
from langchain.schema import BaseOutputParser

class CustomListOutputParser(BaseOutputParser):
    def parse(self, text: str) -> list[str]:
        responses = text.split("---")
        items = [response.strip() for response in responses]
        return items

    def get_format_instructions(self) -> str:
        return '서로 다른 답변은 "---"로 구분하세요.'

# 파서 초기화
response_parser = CustomListOutputParser()

In [11]:
prompt_template = ChatPromptTemplate.from_template("""
다음은 사용자 정보입니다. 이 정보를 바탕으로, 사용자의 성격과 하루 일과, 주요 관심사를를 상상해서 1문단으로 작성하세요.
이를 작성하는 이유는 사용자의 할 일을 사용자의 생활패턴과 맥락에 맞게 구체화하여 추천하기 위해서입니다.
사용자에 대한 이해가 깊어질수록 사용자에게 더 유용한 할 일을 추천할 수 있습니다.
사용자의 긍정적인 면과 부정적인 면을 모두 포함할 수 있도록 작성하세요.

작성된 내용 중 사용자가 적합한 것을 선택할 수 있도록, 서로 다른 내용의 답변을 3~5개 생성하세요.
각각의 답변은 사용자 정보의 다른 부분에 집중하며, 서로 비슷하지 않은 내용이어야 합니다.
예를 들어 한 답변이 "대중교통"이라는 키워드에 집중한다면, 다른 답변은 "도서관" 등 다른 맥락에 집중할 수 있습니다.
만약 비슷한 답변이 생성된다면 생략하세요.

{format_instruction}

사용자 정보: {bio}

답변:
""").partial(format_instruction=response_parser.get_format_instructions())

chain = prompt_template | llm | response_parser
responses = chain.invoke({"bio": user_bio})

In [ ]:
for response in responses:
    print(response)
    print("---")

('윤형석은 서울에 거주하는 30대 초반의 예비창업자로, 내향적이고 직관적인 성격을 지니고 있습니다. 그는 지적 호기심이 강해 새로운 지식을 '
 '탐구하는 것을 즐기지만, 때때로 계획성과 조직력이 부족해 목표 달성에 어려움을 겪기도 합니다. 평일에는 대중교통을 이용해 출퇴근하며, 이 '
 '시간을 활용해 자기계발을 위한 공부를 하거나 유튜브를 통해 최신 트렌드를 접합니다. 주말에는 집에서 애완동물을 돌보며 스트레스를 '
 '해소하고, 카페에서 여유로운 시간을 보내는 것을 좋아합니다. 그러나 가끔은 무기력함에 빠져 취미인 게임이나 영화를 보며 시간을 보내기도 '
 '합니다.')
---
('윤형석은 서울에서 예비창업자로 활동하며, 내향적이고 직관적인 성격을 가지고 있습니다. 그는 평일 출퇴근길에 대중교통을 이용하면서도 시간을 '
 '효율적으로 관리하려고 노력하지만, 계획성 부족으로 인해 가끔은 지연되기도 합니다. 그는 지적 호기심이 강해 새로운 아이디어를 탐구하는 '
 '것을 좋아하지만, 때때로 무기력함을 느껴 목표에 집중하기 어려운 상황에 처하기도 합니다. 주말에는 카페에서 여유롭게 책을 읽거나 유튜브를 '
 '시청하며 재충전하는 시간을 가지며, 이러한 활동을 통해 스트레스를 해소합니다.')
---
('윤형석은 서울에 거주하는 예비창업자로, 내향적이고 직관적인 성격을 지니고 있습니다. 그는 평일 오후에 집이나 도서관에서 집중적으로 '
 '공부하며 지식 확장을 위해 노력하지만, 계획성 부족으로 인해 가끔은 목표를 잊고 방황하기도 합니다. 주말에는 집에서 애완동물을 돌보며 '
 '책임감을 느끼고, 여유로운 시간을 보내는 것을 좋아합니다. 그러나 때때로 무기력함에 빠져 게임이나 영화를 보며 시간을 보내는 경향이 '
 '있어, 이러한 균형을 맞추는 것이 필요합니다.')
---
('윤형석은 서울에서 예비창업자로 활동하며, 내향적이고 직관적인 성격을 지니고 있습니다. 그는 평일 오후에 집에서 공부하며 지적 호기심을 '
 '충족시키는 것을 즐기지만, 때때로 계획성 부족으로

In [13]:
def set_prompt(user: User, responses: list[str], index: int):
    if index < 0 or index >= len(responses):
        raise IndexError("Index out of bounds.")
    user.prompt = responses[index].strip()

set_prompt(user, responses, 1)
user.prompt

'윤형석은 서울에서 예비창업자로 활동하며, 내향적이고 직관적인 성격을 가지고 있습니다. 그는 평일 출퇴근길에 대중교통을 이용하면서도 시간을 효율적으로 관리하려고 노력하지만, 계획성 부족으로 인해 가끔은 지연되기도 합니다. 그는 지적 호기심이 강해 새로운 아이디어를 탐구하는 것을 좋아하지만, 때때로 무기력함을 느껴 목표에 집중하기 어려운 상황에 처하기도 합니다. 주말에는 카페에서 여유롭게 책을 읽거나 유튜브를 시청하며 재충전하는 시간을 가지며, 이러한 활동을 통해 스트레스를 해소합니다.'

In [55]:
class Task(BaseModel):
    name: str
    location_tags: list[str]
    time_tags: list[str]
    other_tags: list[str]
    estimated_minutes: int
    
class Tasks(BaseModel):
    name: str
    context: str
    location_tags: list[str]
    time_tags: list[str]
    other_tags: list[str]
    tasks: list[Task]

tasks_pydantic_parser = PydanticOutputParser(pydantic_object=Tasks)

In [56]:
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()

task_breakdown = ChatPromptTemplate.from_template("""
다음은 사용자가 입력한 헤야 할 일입니다. 이 할 일을 세부적으로 나누어 작성하세요.
각각의 할 일은 최대한 구체적인 행동으로 작성하고, 이를 수행하는 데 필요한 시간과 노력을 고려하여 작성하세요.
할 일을 세부적으로 나누면 사용자가 할 일을 더 쉽게 완료할 수 있습니다.

각각의 할 일은 사용자의 하루를 나타내는 여러 장면을 담은 태그와 함께 저장되고, 사용자가 처한 맥락과 상황을 표현하는 태그에 맞춰 할 일을 추천합니다.
각각의 할 일은 사용자의 위치, 활동하는 장소, 시간대 등을 고려하여 태그를 부여하세요.

Tasks의 context에서는 사용자 정보와 하루 일과에 기반하여 해당 할 일을 언제 하는 것이 좋을지 설명하세요.
각각의 세부 할 일에 적절한 태그를 부여하세요.

{format_instruction}

사용자의 인적 정보: {bio}
사용자의 하루 일과: {prompt}
사용자가 입력한 할 일: {task}
""").partial(format_instruction=tasks_pydantic_parser.get_format_instructions())

task_chain = task_breakdown | llm
task_response = task_chain.invoke({"bio": user_bio, "prompt": user.prompt, "task": "체중 감량을 위한 계획 및 실천하기"})

In [57]:
print(task_response.content)

('```json\n'
 '{\n'
 '  "name": "체중 감량을 위한 계획 및 실천하기",\n'
 '  "context": "윤형석은 체중 감량을 위해 구체적인 계획을 세우고 이를 실천해야 합니다. 평일에는 바쁜 일정을 고려하여 짧은 '
 '시간에 할 수 있는 운동과 식단 조절을 계획하고, 주말에는 여유롭게 운동과 요리를 통해 건강한 식습관을 유지할 수 있습니다.",\n'
 '  "location_tags": ["집", "헬스장", "카페"],\n'
 '  "time_tags": ["평일", "주말", "오전", "오후", "저녁"],\n'
 '  "other_tags": ["건강", "자기계발", "목표 설정"],\n'
 '  "tasks": [\n'
 '    {\n'
 '      "name": "체중 감량 목표 설정",\n'
 '      "location_tags": ["집"],\n'
 '      "time_tags": ["주말", "오후"],\n'
 '      "other_tags": ["목표 설정", "건강"],\n'
 '      "estimated_minutes": 30\n'
 '    },\n'
 '    {\n'
 '      "name": "주간 운동 계획 세우기",\n'
 '      "location_tags": ["집"],\n'
 '      "time_tags": ["주말", "오후"],\n'
 '      "other_tags": ["운동", "계획"],\n'
 '      "estimated_minutes": 30\n'
 '    },\n'
 '    {\n'
 '      "name": "헬스장 가서 운동하기",\n'
 '      "location_tags": ["헬스장"],\n'
 '      "time_tags": ["평일", "오후"],\n'
 '      "other_tags": ["운동", "건강"],\n'
 '      "estimated_minutes": 60\n'
 '    },\n'
 '    {\